In [34]:
import os
import requests
import docx2txt
import docx
from PyPDF2 import PdfReader

from typing import List

import pandas as pd
from langchain.chat_models.gigachat import GigaChat

from langchain.chains import LLMChain

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


from langchain.prompts import load_prompt
from langchain.prompts import PromptTemplate

from getpass import getpass

from openai import OpenAI

In [3]:
class FileConverterService:
    def __init__(self):
        self.file_identifier = None
        self.file_path = None

    def execute(self, file_identifier):
        self.file_identifier = file_identifier
        self.file_path = self._get_file_path()

        if self.file_path:
            return self.convert_to_text()
        else:
            return "File not found"

    def convert_to_text(self):
        file_format = self._get_file_format()

        if file_format == 'doc':
            return self._convert_doc_to_text()
        elif file_format == 'docx':
            return self._convert_docx_to_text()
        elif file_format == 'pdf':
            return self._convert_pdf_to_text()
        else:
            return "Unsupported file format"

    def _get_file_path(self):
        if os.path.isfile(self.file_identifier):
            return self.file_identifier
        elif 'http' in self.file_identifier:
            # If the identifier is a URL, download the file
            file_name = self.file_identifier.split('/')[-1]
            response = self._download_file(self.file_identifier, file_name)

            if response and response.status_code == 200:
                return file_name
            else:
                return None
        else:
            # Assume the identifier is a filename in the current directory
            current_directory = os.getcwd()
            file_path = os.path.join(current_directory, self.file_identifier)

            if os.path.isfile(file_path):
                return file_path
            else:
                return None

    def _download_file(self, url, file_name):
        try:
            response = requests.get(url)
            with open(file_name, 'wb') as file:
                file.write(response.content)
            return response
        except Exception as e:
            print(f"Error downloading file: {str(e)}")
            return None

    def _get_file_format(self):
        if self.file_path.lower().endswith('.doc'):
            return 'doc'
        elif self.file_path.lower().endswith('.docx'):
            return 'docx'
        elif self.file_path.lower().endswith('.pdf'):
            return 'pdf'
        else:
            return None

    def _convert_doc_to_text(self):
        try:
            text = docx2txt.process(self.file_path)
            return text
        except Exception as e:
            return f"Error converting DOC to text: {str(e)}"

    def _convert_docx_to_text(self):
        try:
            with open(self.file_path, 'rb') as file:
                doc = docx.Document(file)
                text = ""
                for para in doc.paragraphs:
                    text += para.text + '\n'
                return text
        except Exception as e:
            return f"Error converting DOCX to text: {str(e)}"

    def _convert_pdf_to_text(self):
        try:
            with open(self.file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                text = ""
                for page_num in range(len(pdf_reader.pages)):
                    text += pdf_reader.pages[page_num].extract_text()
                return text
        except Exception as e:
            return f"Error converting PDF to text: {str(e)}"

# Пример использования
file_identifier = 'Резюме для 1 кейса Хакатона/Alan Abdirasul.pdf'
converter = FileConverterService()
result = converter.execute(file_identifier)

In [36]:
api_key = getpass(prompt='GigaChad Password: ')
gigachad = GigaChat(credentials=api_key, scope="GIGACHAT_API_PERS", verify_ssl_certs=True, model="GigaChat-Pro")

api_key = getpass(prompt='OpenAI Password: ')
openai = OpenAI(
    # temperature=0.0, 
                api_key=api_key)

KeyboardInterrupt: Interrupted by user

In [5]:
class Contact(BaseModel):
    resume_contact_item_id: str = Field(description="ID")
    value: str = Field(description="Значение")
    comment: str = Field(description="Комментарий")
    contact_type: str = Field(description="Тип контакта (1: Телефон, 2: Email, 3: Skype, 4: Telegram, 5: Github)")

class Education(BaseModel):
    resume_education_item_id: str = Field(description="ID")
    year: str = Field(description="Год окончания")
    organization: str = Field(description="Название учебного заведения")
    faculty: str = Field(description="Факультет")
    specialty: str = Field(description="Специальность")
    result: str = Field(description="Результат обучения")
    education_type: str = Field(description="Тип образования", enum = ["Начальное", "Повышение квалификации", "Сертификаты", "Основное"])
    education_level: str = Field(description="Уровень образования", enum = ["Среднее", "Среднее специальное", "Неоконченное высшее", "Высшее, Бакалавр", "Магистр", "Кандидат наук", "Доктор наук"])

class Experience(BaseModel):
    resume_experience_item_id: str = Field(description="ID")
    starts: str = Field(description="Год начала")
    ends: str = Field(description="Год окончания")
    employer: str = Field(description="Организация")
    city: str = Field(description="Город")
    url: str = Field(description="Ссылка на сайт работодателя")
    position: str = Field(description="Должность")
    description: str = Field(description="Описание")
    order: str = Field(description="Порядок следования в массиве опыта работы (для сортировки)") # TUT CHTOT SDELAT NODO

class Language(BaseModel):
    resume_language_item_id: str = Field(description="ID")
    language: str = Field(description="Язык")
    language_level: str = Field(description="Уровень владения языком", enum=["Начальный", "Элементарный", "Средний", "Средне-продвинутый", "Продвинутый", "В совершенстве", "Родной"])

class Resume(BaseModel):
    resume_id: str = Field(description="ID резюме")
    first_name: str = Field(description="Имя")
    last_name: str = Field(description="Фамилия")
    middle_name: str = Field(description="Отчество")
    birth_date: str = Field(description="Дата рождения в формате YYYY-MM-DD")
    birth_date_year_only: bool = Field(description="Если true, дата рождения вычисляется из возраста (Например, возраст 20 -> 2004-01-01)")
    country: str = Field(description="Страна")
    city: str = Field(description="Город")
    about: str = Field(description="Описание")
    key_skills: str = Field(description="Ключевые навыки")
    salary_expectations_amount: str = Field(description="Зарплатные ожидания")
    salary_expectations_currency: str = Field(description="Валюта зарплатных ожиданий")
    photo_path: str = Field(description="Ссылка на фото")
    gender: str = Field(description="Пол {1: Мужской, 2: Женский}")
    resume_name: str = Field(description="Название резюме")
    source_link: str = Field(description="Ссылка на источник резюме")
    contact: List[Contact] = Field(description="Контактные данные")
    education: List[Education] = Field(description="Образование")
    experience: List[Experience] = Field(description="Опыт работы")
    language: List[Language] = Field(description="Владение иностранными языками")


parser = JsonOutputParser(pydantic_object=Resume)


In [6]:
instructions = parser.get_format_instructions()
decoded_instructions = bytes(instructions, "utf-8").decode("unicode_escape")

prompt_template = """Из следующего текста извлеки информацию:

###

text: {text}

###

{format_instructions}

"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["text"],
    partial_variables={"format_instructions": decoded_instructions},
)

In [28]:
chain = prompt | gigachad | parser

chain.invoke({"text": result})

ReadTimeout: The read operation timed out

In [10]:
chain = prompt | openai | parser
chain.invoke({"text": result})

{'resume_id': '123456789',
 'first_name': 'Alan',
 'last_name': 'Abdirasul',
 'middle_name': 'Senior Java Software Engineer',
 'birth_date': '1990-01-01',
 'birth_date_year_only': False,
 'country': 'USA',
 'city': 'Chicago',
 'about': 'Experienced Software Engineer with 5+ years of experience participating in all aspects of the software development lifecycle.',
 'key_skills': 'Java, Kotlin, JavaScript, Spring boot, WebFlux, Gradle, Maven, RabbitMQ, Kafka, WebSocket, Spring Cloud, Hibernate, GraphQL, gRPC, React, Redux, HTML, CSS, Git, Github, Gitlab, CodeCommit, CI/CD, Docker, Kubernetes, AWS',
 'salary_expectations_amount': '100000',
 'salary_expectations_currency': 'USD',
 'photo_path': 'https://example.com/photo.jpg',
 'gender': '1',
 'resume_name': 'Senior Java Software Engineer',
 'source_link': 'https://example.com/resume',
 'contact': [{'resume_contact_item_id': '1',
   'value': 'aаааааааbdirasul@gmail.com',
   'comment': 'Primary Email',
   'contact_type': '2'}],
 'education':

In [20]:
print(prompt.format(text = result))

Из следующего текста извлеки информацию:

###

text: Alan Abdirasul Senior Java Software Engineer |aаааааааbdirasul@gmail.com Summary  Experienced Software Engineer with 5+ years of experience participating in all aspects of the software development lifecycle, which includes estimating, technical design, implementation documentation, testing, deployment, and support of applications developed for various clients. Skills    • Programming Languages: Java, Kotlin, JavaScript, TypeScript • Backend Technologies: Spring boot, WebFlux, Gradle, Maven, RabbitMQ, Kafka, WebSocket, Spring Cloud, Hibernate, GraphQL, gRPC • Databases: PostgreSql, MySql, MongoDB, Redis, Oracle Database, R2DBC • Frontend Technologies: React, Redux, HTML, CSS • Software development tools: Git, Github, Gitlab, CodeCommit, CI/CD, CircleCi, Jenkins, Grafana, Prometheus, Graphite, JUnit, Mockito • Cloud: Docker, Kubernetes, AWS, Digital Ocean • Soft Skills: Leadership, communication, problem-solving, adaptability, teamwork

In [33]:
pip install getpass

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass
Note: you may need to restart the kernel to use updated packages.
